In [2]:
from transformers import pipeline
import mediapipe as mp
from PIL import Image
import numpy as np

c:\Users\salman\OneDrive\Desktop\SEM7\Project-2\Face_Detection\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
class FaceCounter:
    def __init__(self):
        self.mp_face_detection = mp.solutions.face_detection
        self.face_detection = self.mp_face_detection.FaceDetection()

    def count_faces(self, image_path):
        image = Image.open(image_path)
        image_np = self._pil_to_numpy(image)

        results = self.face_detection.process(image_np)
        num_faces = len(results.detections) if results.detections else 0

        return num_faces

    def _pil_to_numpy(self, image):
        return np.array(image)


In [11]:
image_path = "./images/2.jpg"

In [12]:
face_counter = FaceCounter()

In [13]:
total_faces = face_counter.count_faces(image_path)
print(total_faces)

1


In [14]:
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [15]:
base_options = python.BaseOptions()

In [16]:
options = vision.FaceDetectorOptions(base_options=base_options, min_detection_confidence=0.5)

In [17]:
try:
    detector = vision.FaceDetector.create_from_options(options)
except Exception as error:
    print(error)

ExternalFile must specify at least one of 'file_content', 'file_name', 'file_pointer_meta' or 'file_descriptor_meta'.


In [3]:
import cv2

imagePath = './images/4.jpeg'

In [4]:
import cv2
img = cv2.imread(imagePath)

In [20]:
img.shape

(890, 1599, 3)

In [21]:
gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [22]:
gray_image.shape

(890, 1599)

In [5]:
face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

face_classifier_2 = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_alt.xml"
)

face_classifier_3 = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_alt2.xml"
)

face_classifier_4 = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_alt_tree.xml"
)


In [6]:
face = face_classifier.detectMultiScale(
    img, scaleFactor=1.1, minNeighbors=5
)

In [7]:
face

array([[ 638,  130,   88,   88],
       [ 888,  235,   72,   72],
       [ 588,  166,   33,   33],
       [ 623,  285,   89,   89],
       [1493,  295,   78,   78],
       [ 829,  194,   57,   57],
       [ 129,  215,   34,   34],
       [ 531,  211,   50,   50],
       [1241,  214,   47,   47],
       [ 432,  197,   78,   78],
       [1079,  222,   70,   70],
       [ 989,  225,   52,   52]])

In [12]:
for (x, y, w, h) in face:
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 4)

cv2.imshow("Face", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [9]:
import matplotlib.pyplot as plt



plt.figure(figsize=(20,10))
plt.imshow(img_rgb)
plt.axis('off')

NameError: name 'img_rgb' is not defined

<Figure size 2000x1000 with 0 Axes>

In [17]:
import torch
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from PIL import Image, ImageDraw

class FaceDetector:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = fasterrcnn_resnet50_fpn(pretrained=True)
        self.model.to(self.device)
        self.model.eval()

        self.transforms = transforms.Compose([
            transforms.ToTensor()
        ])

    def detect_faces(self, image_path):
        image = Image.open(image_path).convert("RGB")
        image_tensor = self.transforms(image).unsqueeze(0).to(self.device)

        with torch.no_grad():
            predictions = self.model(image_tensor)
        
        return predictions[0]

    def visualize_detections(self, image_path, predictions):
        image = Image.open(image_path)
        draw = ImageDraw.Draw(image)
        
        for score, label, box in zip(predictions["scores"], predictions["labels"], predictions["boxes"]):
            if score > 0.7:  # You can adjust this threshold
                box = [round(i.item()) for i in box]
                draw.rectangle(box, outline="red", width=2)
        
        image.show()


In [18]:
face_detector = FaceDetector()

c:\Users\salman\OneDrive\Desktop\SEM7\Project-2\Face_Detection\env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\salman\OneDrive\Desktop\SEM7\Project-2\Face_Detection\env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:
predictions = face_detector.detect_faces(image_path)

In [16]:
face_detector.visualize_detections(image_path, predictions)

In [20]:
total_faces = len(predictions["scores"])

In [21]:
print(total_faces)

37


In [12]:
image_path = './images/2.jpg'

In [22]:

import cv2
import torch
import numpy as np
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer

class FaceDetector:
    def __init__(self):
        self.cfg = get_cfg()
        self.cfg.merge_from_file("configs/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Change to the appropriate config file
        self.cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
        self.cfg.MODEL.WEIGHTS = "path/to/model/weights"  # Change to the appropriate path
        self.predictor = DefaultPredictor(self.cfg)

    def detect_faces(self, image_path):
        im = cv2.imread(image_path)
        outputs = self.predictor(im)
        return outputs

    def visualize_detections(self, image_path, outputs):
        im = cv2.imread(image_path)
        v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(self.cfg.DATASETS.TRAIN[0]), scale=1.2)
        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        cv2.imshow("Detection", v.get_image()[:, :, ::-1])
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [23]:
face_detector = FaceDetector()

AssertionError: Config file 'configs/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml' does not exist!